사이킷런은 보팅 방식의 앙상블을 구현한 Voting_Classifier 클래스를 제공 

In [1]:
# 로지스틱 회귀와 KNN을 활용 

import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [17]:
cancer = load_breast_cancer()

cancer_df_X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
cancer_df_X

y_target = cancer.target
y_target_res = pd.DataFrame(y_target.reshape(-1,1))

cancer_df = cancer_df_X.copy()
cancer_df['target'] = y_target
cancer_df.head()


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


로지스틱 회귀와 KNN기반의 소프트 보팅 방식으로 보팅 분류기 생성,

사이킷런의 VotingClassifier 클래스는 주요 생성 인자로 estimators와 voting값을 입력받음. estimators는 리스트 값으로 보팅에 사용될 여러 개의 Classifier 객체들을 튜플 형식으로 입력 받으며 voting은
'hard'시 hard voting, 'soft'일시에는 soft voting을 적용하겠다는 의미

In [29]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_ftrs, y_target, test_size=0.2,
                                                   random_state=156)

# 모델들 객체화
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()

# 개별 모델을 soft voting 앙상블 모델로 구현
vo_clf = VotingClassifier(estimators=[('LR', lr_clf),('KNN', knn_clf)], voting='soft')

# VotingClassifier 학습/예측/평가
vo_clf.fit(X_train,y_train)
pred = vo_clf.predict(X_test)
print('Voting 분류기 정확도:{0:.4f}'.format(accuracy_score(y_test,pred)))

# 각 알고리즘별 코드 비교
clfs = [lr_clf, knn_clf]

for clf in clfs:
    clf.fit(X_train,y_train)
    pred = clf.predict(X_test)
    clf_name = clf.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(clf_name,accuracy_score(y_test,pred)))

Voting 분류기 정확도:0.9474
LogisticRegression 정확도: 0.9386
KNeighborsClassifier 정확도: 0.9035


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

voting 분류기가 정확도가 좀 더 높게 나왔으나, voting으로 여러개의 learners를 결합한다고 해서 반드시 예측 정확도가 향상된다라고 하기는 어렵다. 데이터의 특성, 분포 등 다양한 상황에 따라 오히려 단일 learner가 더 좋은 결과를 나타내는 경우도 많다.

그럼에도 불구하고 voting, bagging, boosting 등의 앙상블 방법은 전반적으로 다른 단일 ML_algorithm 보다 뛰어난 예측 성능을 가지는 경우가 더 많다. 

    1) voting & stacking = 서로 다른 알고리즘으로 진행됨
    
    2) bagging & boosting = 의사결정나무_알고리즘(DecisionTree)

In [30]:
# end of file